# Quantitative Analysis and Backtesting Project
## Magnificent 7 Backtesting

This project involves creating a backtesting software to evaluate the performance of the "Magnificent 7" stocks (MSFT, AAPL, NVDA, AMZN, GOOG, META, TSLA) using the Double Bollinger Bands strategy.


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

We will begin by downloading historical price data for the Magnificent 7 stocks from the Yahoo Finance API using the `yfinance` library. The historical data will cover the period from January 1, 2013, to December 31, 2023.



In [2]:
tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']

start_date = '2013-01-01'
end_date = '2023-12-31'

data_frames = []

# Fetch and store data for each ticker
for ticker in tickers:
    df = yf.download(ticker, start=start_date, end=end_date)
    df['Ticker'] = ticker
    data_frames.append(df)

# Combine individual DataFrames into one and adjust the index
combined_data = pd.concat(data_frames)
combined_data.reset_index(inplace=True)

combined_data.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2013-01-02,27.250000,27.730000,27.150000,27.620001,22.492289,52899300,MSFT
1,2013-01-03,27.629999,27.650000,27.160000,27.250000,22.190968,48294400,MSFT
2,2013-01-04,27.270000,27.340000,26.730000,26.740000,21.775658,52521100,MSFT
3,2013-01-07,26.770000,26.879999,26.639999,26.690001,21.734940,37110400,MSFT
4,2013-01-08,26.750000,26.790001,26.459999,26.549999,21.620926,44703100,MSFT


## Calculate Double Bollinger Bands
In this section, we calculate the Double Bollinger Bands (DBB) for each of the Magnificent 7 stocks. DBB is a variation of the standard Bollinger Bands and includes two sets of bands: one set at two standard deviations away from a 20-period moving average, and another set at one standard deviation. This approach helps in identifying potential buy and sell signals based on the distance of the stock price from these bands.

Steps to Calculate Double Bollinger Bands:
1. Calculate the 20-period moving average (MA20) of the stock's closing prices.
2. Determine the standard deviation (STD20) over the same period.
3. Create four additional bands:
 - Upper Band 2 (UB2): MA20 + (2 * STD20)
 - Lower Band 2 (LB2): MA20 - (2 * STD20)
 - Upper Band 1 (UB1): MA20 + (1 * STD20)
 - Lower Band 1 (LB1): MA20 - (1 * STD20)

These bands will serve as thresholds to assess whether the stock is in a high or low volatility phase, providing insights into potential market movements.

In [3]:
def calculate_double_bollinger_bands(data, window=20):
    # Calculate the 20-period moving average (MA20) and standard deviation (STD20)
    data['MA20'] = data['Close'].rolling(window=window).mean()
    data['STD20'] = data['Close'].rolling(window=window).std()

    # Create the four additional bands
    data['UpperBand_2'] = data['MA20'] + (data['STD20'] * 2)
    data['LowerBand_2'] = data['MA20'] - (data['STD20'] * 2)
    data['UpperBand_1'] = data['MA20'] + (data['STD20'] * 1)
    data['LowerBand_1'] = data['MA20'] - (data['STD20'] * 1)
    return data

combined_data = combined_data.groupby('Ticker', group_keys=False).apply(calculate_double_bollinger_bands).reset_index(drop=True)
print(combined_data.iloc[20:40][['Date', 'Ticker', 'Close', 'MA20', 'STD20', 'UpperBand_2', 'LowerBand_2', 'UpperBand_1', 'LowerBand_1']].to_string(index=False))


      Date Ticker     Close    MA20    STD20  UpperBand_2  LowerBand_2  UpperBand_1  LowerBand_1
2013-01-31   MSFT 27.450001 27.2175 0.482045    28.181589    26.253410    27.699545    26.735455
2013-02-01   MSFT 27.930000 27.2515 0.507753    28.267007    26.235993    27.759253    26.743747
2013-02-04   MSFT 27.440001 27.2865 0.494595    28.275690    26.297310    27.781095    26.791905
2013-02-05   MSFT 27.500000 27.3270 0.475993    28.278987    26.375013    27.802993    26.851007
2013-02-06   MSFT 27.340000 27.3665 0.439501    28.245501    26.487499    27.806001    26.926999
2013-02-07   MSFT 27.280001 27.3955 0.411448    28.218396    26.572604    27.806948    26.984052
2013-02-08   MSFT 27.549999 27.4500 0.348365    28.146729    26.753271    27.798365    27.101635
2013-02-11   MSFT 27.860001 27.5015 0.327386    28.156273    26.846727    27.828886    27.174114
2013-02-12   MSFT 27.879999 27.5510 0.304076    28.159152    26.942849    27.855076    27.246924
2013-02-13   MSFT 28.030001 27

## Generating buy and sell signals

To capitalize on market opportunities, we generate buy and sell signals based on the stock's closing price relative to its Double Bollinger Bands. The strategy defines two zones for action:

**Buy Zone:** When the stock's closing price moves between the first and second upper Bollinger Bands (indicative of a strong upward momentum but not overbought).

**Sell Zone:** When the stock's closing price is between the first and second lower Bollinger Bands (indicative of strong downward momentum but not oversold).

This section details the implementation of this strategy by defining conditions under which to signal a buy or sell.

In [4]:
def generate_signals(data):
    data['BuySignal'] = 0
    data['SellSignal'] = 0

    # Buy signal: Price moves into the buy zone (between UpperBand_1 and UpperBand_2)
    data.loc[(data['Close'] > data['UpperBand_1']) & (data['Close'] <= data['UpperBand_2']), 'BuySignal'] = 1

    # Sell signal: Price moves into the sell zone (between LowerBand_1 and LowerBand_2)
    data.loc[(data['Close'] < data['LowerBand_1']) & (data['Close'] >= data['LowerBand_2']), 'SellSignal'] = 1

    return data

combined_data = combined_data.groupby('Ticker', group_keys=False).apply(generate_signals).reset_index(drop=True)
print(combined_data.iloc[20:40][['Date', 'Ticker', 'Close', 'BuySignal', 'SellSignal', 'MA20', 'STD20', 'UpperBand_2', 'LowerBand_2', 'UpperBand_1', 'LowerBand_1']].to_string(index=False))


      Date Ticker     Close  BuySignal  SellSignal    MA20    STD20  UpperBand_2  LowerBand_2  UpperBand_1  LowerBand_1
2013-01-31   MSFT 27.450001          0           0 27.2175 0.482045    28.181589    26.253410    27.699545    26.735455
2013-02-01   MSFT 27.930000          1           0 27.2515 0.507753    28.267007    26.235993    27.759253    26.743747
2013-02-04   MSFT 27.440001          0           0 27.2865 0.494595    28.275690    26.297310    27.781095    26.791905
2013-02-05   MSFT 27.500000          0           0 27.3270 0.475993    28.278987    26.375013    27.802993    26.851007
2013-02-06   MSFT 27.340000          0           0 27.3665 0.439501    28.245501    26.487499    27.806001    26.926999
2013-02-07   MSFT 27.280001          0           0 27.3955 0.411448    28.218396    26.572604    27.806948    26.984052
2013-02-08   MSFT 27.549999          0           0 27.4500 0.348365    28.146729    26.753271    27.798365    27.101635
2013-02-11   MSFT 27.860001          1  

## Backtesting the Double Bollinger Band Strategy
This section focuses on backtesting the Double Bollinger Band strategy with an initial investment capital of $10,000. We simulate trading actions based on the generated buy and sell signals to evaluate the strategy's potential returns.

The backtest calculates the portfolio value dynamically as trading signals dictate buying or selling actions.

Backtesting Steps:
1. Initialize Capital: Start with $10,000 for each stock.
2. Simulate Trades: Iterate over each stock's data:
 - Buy: If a buy signal is detected and sufficient funds are available, buy as many shares as possible without exceeding the available cash.
 - Sell: If a sell signal is detected, sell all holdings of the stock at the current price.
3. Track Portfolio Value: Update cash reserves and the position for each transaction, and calculate the total portfolio value (cash + current stock holdings).

In [5]:
initial_capital = 10000
combined_data['Position'] = 0
combined_data['Cash'] = initial_capital
combined_data['PortfolioValue'] = initial_capital

for ticker in tickers:
    ticker_data = combined_data[combined_data['Ticker'] == ticker].copy()
    cash = initial_capital
    position = 0

    # Iterate through each day's data after the first day
    for i in range(1, len(ticker_data)):
        # Check for buy signal
        if ticker_data['BuySignal'].iloc[i] == 1 and cash > 0:
            num_shares = cash // ticker_data['Close'].iloc[i]
            cost = num_shares * ticker_data['Close'].iloc[i]
            cash -= cost
            position += num_shares
        # Check for sell signal
        elif ticker_data['SellSignal'].iloc[i] == 1 and position > 0:
            proceeds = position * ticker_data['Close'].iloc[i]
            cash += proceeds
            position = 0

        combined_data.loc[ticker_data.index[i], 'Cash'] = cash
        combined_data.loc[ticker_data.index[i], 'Position'] = position
        combined_data.loc[ticker_data.index[i], 'PortfolioValue'] = cash + (position * ticker_data['Close'].iloc[i])

print(combined_data.iloc[20:40][['Date', 'Ticker', 'Close', 'Position', 'Cash', 'PortfolioValue']].to_string(index=False))


      Date Ticker     Close  Position        Cash  PortfolioValue
2013-01-31   MSFT 27.450001       359    1.849863     9856.400137
2013-02-01   MSFT 27.930000       359    1.849863    10028.719973
2013-02-04   MSFT 27.440001       359    1.849863     9852.810055
2013-02-05   MSFT 27.500000       359    1.849863     9874.349863
2013-02-06   MSFT 27.340000       359    1.849863     9816.909918
2013-02-07   MSFT 27.280001       359    1.849863     9795.370110
2013-02-08   MSFT 27.549999       359    1.849863     9892.299589
2013-02-11   MSFT 27.860001       359    1.849863    10003.590082
2013-02-12   MSFT 27.879999       359    1.849863    10010.769562
2013-02-13   MSFT 28.030001       359    1.849863    10064.620110
2013-02-14   MSFT 28.040001       359    1.849863    10068.210192
2013-02-15   MSFT 28.010000       359    1.849863    10057.439945
2013-02-19   MSFT 28.049999       359    1.849863    10071.799589
2013-02-20   MSFT 27.870001       359    1.849863    10007.180164
2013-02-21

## Compute Performance Metrics
To assess the effectiveness of the Double Bollinger Band strategy, we calculate various performance metrics.

### Total Return
- **What it measures:** The total percentage change in the portfolio from the beginning to the end of the period.
- **How to calculate:**
  Total Return = (Final Portfolio Value / Initial Capital) - 1  
  This calculation shows how much the initial investment has grown or shrunk.

### Annual Return
- **What it measures:** The average amount of money earned by the investment each year.
- **How to calculate:**
  Annual Return = (1 + Mean Daily Return) ^ 252 - 1  
  The mean daily return is annualized to reflect what the return would be over a year, assuming 252 trading days in a year.

### Annual Volatility
- **What it measures:** The standard deviation of the investment's daily returns, representing the risk or variability in the investment's value over a year.
- **How to calculate:**
  Annual Volatility = Standard Deviation of Daily Returns * sqrt(252)  
  This scales the daily standard deviation to an annual measure.

### Sharpe Ratio
- **What it measures:** The average return earned in excess of the risk-free rate per unit of volatility or total risk.
- **How to calculate:**
  Sharpe Ratio = (Annual Return - Risk-Free Rate) / Annual Volatility  
  It provides a way to compare the return of an investment with its risk. A higher Sharpe ratio means better return for the same risk.  

  For the Risk_Free Rate, I will be using the average rate from the 3-month treasury bill secondary market rate from January 1, 2013, to December 31, 2023. The average obtained from FRED data is 1.20%

### Sortino Ratio
- **What it measures:** Similar to the Sharpe Ratio, but it only considers downside or negative volatility, which is more relevant to most investors.
- **How to calculate:**
  Sortino Ratio = (Expected Annual Return - Risk-Free Rate) / Downside Deviation  
  The downside deviation is the standard deviation of negative asset returns.

### Maximum Drawdown
- **What it measures:** The largest single drop from peak to bottom in the value of a portfolio, before a new peak is achieved.
- **How to calculate:**
  Maximum Drawdown = min(Portfolio Value / Rolling Maximum Portfolio Value - 1)  
  It is found by taking the maximum of the largest drop from a peak to a trough in the portfolio value, based on a cumulative return series.


In [6]:
def calculate_total_return(final_portfolio_value, initial_capital):
    return (final_portfolio_value / initial_capital) - 1

def calculate_annual_return(daily_returns):
    return (1 + daily_returns.mean()) ** 252 - 1

def calculate_annual_volatility(daily_returns):
    return daily_returns.std() * np.sqrt(252)

def calculate_sharpe_ratio(annual_return, annual_volatility, risk_free_rate=0.012):
    return (annual_return - risk_free_rate) / annual_volatility

def calculate_sortino_ratio(returns, risk_free_rate=0.012):
    downside_returns = returns[returns < 0]
    expected_return = returns.mean() * 252
    downside_deviation = downside_returns.std() * np.sqrt(252)
    return (expected_return - risk_free_rate) / downside_deviation

def calculate_max_drawdown(portfolio_values, window=252):
    roll_max = portfolio_values.rolling(window, min_periods=1).max()
    daily_drawdown = portfolio_values / roll_max - 1.0
    max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()
    return max_daily_drawdown.min()

total_return = calculate_total_return(combined_data['PortfolioValue'].iloc[-1], initial_capital)
daily_returns = combined_data['PortfolioValue'].pct_change().dropna()
annual_return = calculate_annual_return(daily_returns)
annual_volatility = calculate_annual_volatility(daily_returns)
sharpe_ratio = calculate_sharpe_ratio(annual_return, annual_volatility)
sortino_ratio = calculate_sortino_ratio(daily_returns)
max_drawdown = calculate_max_drawdown(combined_data['PortfolioValue'])

print(f"Total Return: {total_return:.2%}")
print(f"Annual Return: {annual_return:.2%}")
print(f"Annual Volatility: {annual_volatility:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino_ratio:.2f}")
print(f"Maximum Drawdown: {max_drawdown:.2%}")

Total Return: 12775.16%
Annual Return: 18.72%
Annual Volatility: 34.72%
Sharpe Ratio: 0.50
Sortino Ratio: 0.34
Maximum Drawdown: -96.04%
